In [1]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from fh_filter import get_filter
import os
from supabase import create_client, Client
import math
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [ ]:
def mount_gols_filters(fh_q_dict, dict_campo_amostral):
    category = 'gols'
    gol_atual = fh_q_dict['gols']['sum_period1']
    ca_category = dict_campo_amostral['gols'] 
    print(gol_atual)
    dict_gols_filter_over = {
        1 :  [gol_atual + 0.5, get_filter(fh_q_dict, category, 1),0,0],
        2 : [gol_atual + 1.5, get_filter(fh_q_dict, category, 2), 0,0],
        3 : [gol_atual + 2.5, get_filter(fh_q_dict, category, 3), 0,0]
    }

    dict_gols_filter_under = {
        1 : [gol_atual + 0.5 ,ca_category - get_filter(fh_q_dict, category, 1), 0, 0],
        2 : [gol_atual + 1.5, ca_category - get_filter(fh_q_dict, category, 2),0, 0],
        3 : [gol_atual + 2.5, ca_category - get_filter(fh_q_dict, category, 3),0, 0]
    }


    return dict_gols_filter_over, dict_gols_filter_under


def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    
    for i in range(1,len(dict_gols_filter_over) + 1):
        filter = dict_gols_filter_over[i][1]
        prob = (filter * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob


    for i in range(1,len(dict_gols_filter_under) + 1):
        filter = dict_gols_filter_under[i][1]
        prob = (filter * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob

    return dict_gols_filter_over, dict_gols_filter_under


def mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under):
    for i in range(1,len(dict_gols_filter_over) + 1):
        prob = (dict_gols_filter_over[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_over[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_over[i][3] = odd

    for i in range(1,len(dict_gols_filter_under) + 1):
        prob = (dict_gols_filter_under[i][1] * 100) / dict_campo_amostral['gols']
        dict_gols_filter_under[i][2] = prob
        odd = 1/(prob/100)
        dict_gols_filter_under[i][3] = odd

    return dict_gols_filter_over, dict_gols_filter_under




In [4]:

event_id = 14064418
dict_campo_amostral, fh_q_dict = campo_amostral(event_id)
# dict_campo_amostral, fh_q_dict

In [6]:
dict_gols_filter_over, dict_gols_filter_under = mount_gols_filters(fh_q_dict, dict_campo_amostral)

0
145


{1: [0.5, 118, 0, 0], 2: [1.5, 64, 0, 0], 3: [2.5, 32, 0, 0]}

{1: [0.5, 27, 0, 0], 2: [1.5, 81, 0, 0], 3: [2.5, 113, 0, 0]}

In [7]:
mount_gols_resp(dict_gols_filter_over, dict_gols_filter_under)

({1: [0.5, 118, 81.37931034482759, 1.228813559322034],
  2: [1.5, 64, 44.13793103448276, 2.2656249999999996],
  3: [2.5, 32, 22.06896551724138, 4.531249999999999]},
 {1: [0.5, 27, 18.620689655172413, 5.370370370370371],
  2: [1.5, 81, 55.86206896551724, 1.7901234567901234],
  3: [2.5, 113, 77.93103448275862, 1.2831858407079646]})